# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
weather_data_to_load = "CityWeather.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,21,PF,1590384545,72,-23.12,-134.97,72.09,25.26
1,Tuktoyaktuk,20,CA,1590384545,74,69.45,-133.04,37.40,14.99
2,Coro,70,VE,1590384545,81,11.40,-69.67,77.92,15.05
3,Xapuri,43,BR,1590384545,75,-10.65,-68.50,60.30,2.42
4,Busselton,95,AU,1590384292,83,-33.65,115.33,53.01,29.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

# # Plot Heatmap
fig = gmaps.figure()

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
vacation_cities = weather_data.loc[(weather_data['Cloudiness'] <= 15) \
                                   & (weather_data['Wind Speed'] < 15) \
                                  & (weather_data['Max Temp'] < 85) & (weather_data['Max Temp'] > 65) \
                                  & (weather_data['Humidity'] < 30)]

vacation_cities.dropna()

vacation_cities.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,Jalu,0,LY,1590384549,24,29.03,21.55,69.82,9.31
316,Goundam,0,ML,1590384617,16,16.41,-3.67,80.96,7.94
328,Pingzhuang,11,CN,1590384619,22,42.04,119.29,70.63,9.98
338,Sabzawār,0,IR,1590384622,15,36.21,57.68,84.79,7.25
367,Changji,5,CN,1590384628,24,44.02,87.32,78.80,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [83]:
# 04.02.06 copy/drop
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html


hotel_df = vacation_cities[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''

# part II a dataframe is created
# that contains the following:
# ✓ Ten or less rows.
# ✓ City
# ✓ Country
# ✓ Latitude
# ✓ Longitude
# ✓ Hotel Name

In [90]:

# Set up a parameters dictionary
params = {
    'radius': 5000, 
    'type': 'lodging',
#     'keyword': 'Hotel',
    'key': g_key
}



# Place Search
# https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
# 6.3.5
#   Loop through the hotels_df and run a lat/long search to ID nearby hotels
for index, row in hotel_df.iterrows():
    
          
    lat = row['Lat']
    lng = row['Lng']
         
         #update location key value
         # create the 'location' parameter for our request
    params['location'] = f"{lat}, {lng}"
    
    # Base URL - Nearby search
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
         
    # Make request
    hotel_name = requests.get(base_url, params=params)
         
         # convert to json / redefine to just contain the data
    hotel_name = hotel_name.json()
        
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except:
        print('Missing field/result... skipping.')
        
        
    
# run a request using paramaters dictionary
# response = requests.get(base_url, params=params)

# test_req   
# try: 
  

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [91]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
21,Jalu,LY,29.03,21.55,Jalu Hotel
316,Goundam,ML,16.41,-3.67,
328,Pingzhuang,CN,42.04,119.29,
338,Sabzawār,IR,36.21,57.68,پارکینگ وسایل سنگین جنوب
367,Changji,CN,44.02,87.32,Lvye Holiday Resort
372,Fuxin,CN,42.02,121.66,Xinya International Hotel
410,Kungurtug,RU,50.60,97.52,
420,Tahe,CN,52.33,124.73,塔河宾馆
486,Tagāb,AF,34.85,69.65,


In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
#use hotel name, city, country - ref 6.3.6

markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig


# Display figure - 


Figure(layout=FigureLayout(height='420px'))